In [1]:
import os
from googletrans import Translator
import whisper
from pytube import YouTube
from pydub import AudioSegment
import cutlet
from lyrics_scrap import scrap
import pickle
from pathlib import Path
import comparison as cp
import pykakasi

In [20]:
katsu = cutlet.Cutlet(use_foreign_spelling=False)
katsu.romaji(" 私")

'Watakushi'

In [2]:
song_info = [
    ('藍二乗', 'ヨルシカ'), 
    ('言って', 'ヨルシカ')
]
song_idx = 1

In [4]:
def save_mp3(video_url, folder_path, song_name):
    print(f"saving mp3 for {song_name}")
    yt = YouTube(video_url)
    aud = yt.streams.filter(only_audio=True).first()
    temp = aud.download(filename='temp_audio.mp3')
    audio = AudioSegment.from_file(temp)
    audio.export(f"{folder_path}/{song_name}.mp3", format='mp3')
    os.remove(temp)
    
def save_lyrics(song_name, song_composer, folderpath) -> str:
    print(f"saving lyrics for {song_name}")
    lyrics, yt_link = scrap(song_name, song_composer)
    
    with open(f"{folderpath}/{song_name}.pickle", 'wb') as fw:
        pickle.dump(lyrics, fw)
    return yt_link

targ_path = '../../mfvr/target'
file_names = [f.name for f in Path(targ_path).iterdir() if f.is_file()]
for name, composer in song_info:
    if f"{name}.pickle" not in file_names: 
        link = save_lyrics(name, composer, targ_path)
    if f"{name}.mp3" not in file_names:
        save_mp3(link, targ_path, name)  

In [6]:
def rgbtxt(trgb=None, brgb=None, text="Default text"):
    tt = "" if trgb==None else f"\033[38;2;{trgb[0]};{trgb[1]};{trgb[2]}m"
    bt = "" if brgb==None else f"\033[48;2;{brgb[0]};{brgb[1]};{brgb[2]}m"
    return f"{tt}{bt}{text}\033[0m"

In [5]:
def whispering(audio_path, song_name):
    arg = dict(word_timestamps=True, condition_on_previous_text=False
               , language="Japanese"
               , initial_prompt=f"Song name is {song_name}.")
    model = whisper.load_model("medium")
    result = model.transcribe(audio_path, **arg)
    
    s = dict(rom=[], jap=[], startstamp=[], endstamp=[], rom_ly='', jap_ly='', prb=[])
    # katsu = cutlet.Cutlet(use_foreign_spelling=False)
    kks = pykakasi.kakasi()
    for d in result['segments']:
        result = kks.convert(d['text'])
        result = [r for r in result if r['orig'] != ' ']  # 공백 제거
        temp_rom = [r['hepburn'] for r in result]; temp_jap = [r['orig'] for r in result]
        acc=0; idx=0; prbsum=0
        for pt in d['words']:
            pt_len = len(pt['word']); dt = (pt['end'] - pt['start']) / pt_len
            for i in range(pt_len):
                st = pt['start'] + dt*i; ed = st + dt; prbsum += pt['probability']
                if acc == len(s['startstamp']): s['startstamp'].append(st)
                idx+=1
                if idx == len(temp_jap[acc]):
                    s['prb'] = prbsum / idx; acc+=1; s['endstamp'].append(ed)
                    idx=0; prbsum=0
        s['rom'] += temp_rom; s['jap'] += temp_jap
            
        # for pt in d['words']:
        #     s['rom'].append(katsu.romaji(pt['word'], capitalize=False))
        #     s['jap'].append(pt['word'].strip())
        #     s['timestamp'].append((pt['start'], pt['end']))
        #     s['prb'].append(pt['probability'])
    s['jap_ly'] = ''.join(s['jap'])
    s['rom_ly'] = ''.join(s['rom'])
        
    with open(f"{targ_path}/{song_info[song_idx][0]}_gen.pickle", 'wb') as fw:
        pickle.dump(s, fw)  
    return result

rdict = whispering(f"{targ_path}/{song_info[song_idx][0]}.mp3", song_info[song_idx][0])

c:\Users\naniri\anaconda3\envs\vocal\lib\site-packages\whisper\timing.py:42: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower median kernel implementation...
  warnings.warn(
c:\Users\naniri\anaconda3\envs\vocal\lib\site-packages\whisper\timing.py:146: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower DTW implementation...
  warnings.warn(
c:\Users\naniri\anaconda3\envs\vocal\lib\site-packages\whisper\timing.py:42: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower median kernel implementation...
  warnings.warn(
c:\Users\naniri\anaconda3\envs\vocal\lib\site-packages\whisper\timing.py:146: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower DTW implementation...
  warnings.warn(


IndexError: list index out of range

In [10]:
arg = dict(word_timestamps=True, condition_on_previous_text=False
            , language="Japanese"
            , initial_prompt=f"Song name is {song_info[song_idx][0]}.")
model = whisper.load_model("medium")
result = model.transcribe(f"{targ_path}/{song_info[song_idx][0]}.mp3", **arg)

c:\Users\naniri\anaconda3\envs\vocal\lib\site-packages\whisper\timing.py:42: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower median kernel implementation...
  warnings.warn(
c:\Users\naniri\anaconda3\envs\vocal\lib\site-packages\whisper\timing.py:146: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower DTW implementation...
  warnings.warn(


In [24]:
s = dict(rom=[], jap=[], startstamp=[], endstamp=[], rom_ly='', jap_ly='', prb=[])
kks = pykakasi.kakasi()
for d in result['segments']:
    kksed = kks.convert(d['text'])
    kksed = [r for r in kksed if r['orig'] != ' ']  # 공백 제거
    temp_rom = [r['hepburn'] for r in kksed]; temp_jap = [r['orig'] for r in kksed]
    acc=0; idx=0; prbsum=0
    for pt in d['words']:
        pt['word'] = pt['word'].replace(' ', '')
        pt_len = len(pt['word']); dt = (pt['end'] - pt['start']) / pt_len
        for i in range(pt_len):
            st = pt['start'] + dt*i; ed = st + dt; prbsum += pt['probability']
            # print(pt['word'][i], acc)
            if acc == len(s['startstamp']): s['startstamp'].append(st)
            idx+=1
            if idx == len(temp_jap[acc]):
                s['prb'].append(prbsum / idx); acc+=1; s['endstamp'].append(ed)
                idx=0; prbsum=0
    s['rom'] += temp_rom; s['jap'] += temp_jap
s['jap_ly'] = ''.join(s['jap'])
s['rom_ly'] = ''.join(s['rom'])

In [25]:
s

{'rom': ['anone',
  'watashi',
  'jitsuha',
  'kizutsu',
  'iterundakedo',
  '',
  '',
  '',
  'hora',
  'kun',
  'ga',
  'itta',
  'koto',
  'amai',
  'kangae',
  'taito',
  'omoe',
  'nakute',
  'wasure',
  'tetandakedo',
  'moumokuteki',
  'ni',
  'mousou',
  'tekini',
  'mousou',
  'tekini',
  'iki',
  'te',
  'shoudouteki',
  'na',
  'shousou',
  'tekina',
  'shoukyokuteki',
  'namamaja',
  'dame',
  'dattanda',
  'kitto',
  'jinsei',
  'saikou',
  'no',
  'nichi',
  'wo',
  'mae',
  'ni',
  'omou',
  'nodarou',
  'zenbu',
  'zenbu',
  'itai',
  'nodesukedo',
  'aa',
  ',',
  'itsuka',
  'jinsei',
  'saikou',
  'no',
  'nichi',
  'kun',
  'gainaikotowo',
  'mottomottomotto',
  'mottochanto',
  'itte',
  '(',
  'uta',
  ')',
  'anone',
  'sora',
  'ga',
  'aoi',
  'nottedouyatte',
  'tsutae',
  'rebaiindaroune',
  'yoru',
  'no',
  'kumo',
  'ga',
  'takai',
  'nottedousureba',
  'kun',
  'mowakarundarou',
  'itte',
  'anone',
  'watashi',
  'jitsuha',
  'waka',
  'tteruno',
  'mou

In [14]:
s['startstamp']

[23.919999999999998,
 27.62,
 27.880000000000003,
 29.060000000000002,
 29.64,
 29.98]

---

In [22]:
with open(f"{targ_path}/{song_info[song_idx][0]}.pickle", 'rb') as fr:
    org_ly = pickle.load(fr)
with open(f"{targ_path}/{song_info[song_idx][0]}_gen.pickle", 'rb') as fr:
    gen_ly = pickle.load(fr)

In [23]:
for l in org_ly:
    print(l)

['言', 'って', '/', '/', 'あのね', '、', '私', '実', 'は', '気付', 'いてるの', '/', 'ほら', '、', '君', 'がいったこと', '/', '/', 'あまり', '考', 'えたいと', '思', 'えなくて', '/', '忘', 'れてたんだけど', '/', '/', '盲目的', 'に', '盲動', '的', 'に', '妄想的', 'に', '生', 'きて', '/', '衝動的', 'な', '焦燥', '的', 'な', '/', '消極的', 'なままじゃ', '駄目', 'だったんだ', '/', '/', 'きっと', '、', '人生', '最後', 'の', '日', 'を', '前', 'に', '思', 'うのだろう', '/', '全部', '、', '全部', '言', 'い', '足', 'りなくて', '惜', 'しいけど', '/', 'あぁ', '、', 'いつか', '人生', '最後', 'の', '日', '、', '/', '君', 'がいないことを', '/', 'もっと', '、', 'もっと', '、', 'もっと', '/', 'もっと', '、', 'ちゃんと', '言', 'って', '/', '/', 'あのね', '、', '空', 'が', '青', 'いのって', '/', 'どうやって', '伝', 'えればいいんだろうね', '/', '夜', 'の', '雲', 'が', '高', 'いのって', '/', 'どうすれば', '君', 'もわかるんだろう', '/', '/', '言', 'って', '/', '/', 'あのね', '、', '私', '実', 'はわかってるの', '/', 'もう', '君', 'が', '逝', 'ったこと', '/', '/', 'あのね', '、', 'わからず', '屋', 'って', '言', 'うんだろうね', '/', '忘', 'れたいんだけど', '/', '/', 'もっとちゃんと', '言', 'ってよ', '/', '忘', 'れないよう', 'メモ', 'にしてよ', '/', '明日', '十時', 'に', 'ホ', 'ー', 'ム', 'で', '待', 'ち'

In [12]:
rdict['segments']

[{'id': 0,
  'seek': 0,
  'start': 23.919999999999998,
  'end': 29.98,
  'text': 'あのね 私 実は傷ついてるんだけど',
  'tokens': [50364,
   24910,
   5555,
   220,
   20083,
   220,
   33197,
   3065,
   41323,
   9335,
   18549,
   4895,
   26983,
   21891,
   50764],
  'temperature': 0.0,
  'avg_logprob': -0.7813610002106312,
  'compression_ratio': 1.8571428571428572,
  'no_speech_prob': 0.03894290328025818,
  'words': [{'word': 'あの',
    'start': 23.919999999999998,
    'end': 25.22,
    'probability': 0.057536784559488297},
   {'word': 'ね',
    'start': 25.22,
    'end': 26.52,
    'probability': 0.9411863684654236},
   {'word': ' 私',
    'start': 27.62,
    'end': 28.14,
    'probability': 0.7814905643463135},
   {'word': ' 実',
    'start': 28.92,
    'end': 29.2,
    'probability': 0.9934095144271851},
   {'word': 'は',
    'start': 29.2,
    'end': 29.64,
    'probability': 0.9983443021774292},
   {'word': '傷',
    'start': 29.64,
    'end': 29.98,
    'probability': 0.8537088632583618},
   {'w

In [30]:
for c in rdict['segments']:
    print(c['text'])

あのね 私 実は傷ついてるんだけど



ほら君が言ったこと
甘い
考えたいと思えなくて
忘れてたんだけど
盲目的に 妄想的に 妄想的に 生きて
衝動的な 焦燥的な 消極的なままじゃ ダメだったんだ
きっと
人生最高の日を前に思うのだろう
全部全部痛いのですけど
ああ、いつか
人生最高の日
君がいないことを
もっともっともっと
もっとちゃんと言って
【歌】
あのね
空が青いのってどうやって
伝えればいいんだろうね
夜の 雲が 高いのってどうすれば
君もわかるんだろう 言って
あのね 私 実は分かってるの もう君が言ったこと
あのね わからずやって言うんだろうね 忘れたいんだけど
もっとちゃんと言ってよ 忘れないよメモにしてよ
明日10時にホームで待ち合わせとかしよう
歌うわじっても花だ
夏が去っても水墨は切な
口に出して
恋に出して
君が言って
そして人生最高の日
君が見えるのなら
きっと 人生最後の日も愛を歌うのだろう
全部 全部 見出せなかったって言うから
ああ いつか 人生最後の日
君みたいないことが まだ信じられないけど
もっと もっと もっと
もっと もっと もっと
君が もっと もっと もっと もっと もっと
ちゃんと 言って


In [7]:
def text_to_romaji(lyrics):
    katsu = cutlet.Cutlet(use_foreign_spelling=False); l = []
    for phrase in lyrics:
        l.append(katsu.romaji(phrase['text'], capitalize=False))
    return l

romaji_list = text_to_romaji(rdict['segments'])
print(romaji_list)

['ano ne watakushi jitsu wa kizutsuiteru n dakedo', '', '', '', 'horakun ga itta koto', 'amai', 'kangaetai to omoenakute', 'wasureteta n dakedo', 'moumokuteki ni mousouteki ni mousouteki ni ikite', 'shoudouteki na shousouteki na shoukyokuteki na mama ja dame datta n da', 'kitto', 'jinsei saikou no hi wo mae ni omou no darou', 'zenbu zenbu itai no desukedo', 'aa, itsu ka', 'jinsei saikou no hi', 'kimi ga inai koto wo', 'motto motto motto', 'motto chanto itte', '[uta]', 'ano ne', 'sora ga aoi no tte dou yatte', 'tsutaereba ii n darou ne', 'yoru no kumo ga takai no tte dou sureba', 'kimi mo wakaru n darou itte', 'ano ne watakushi jitsu wa wakatteru no mou kimi ga itta koto', 'ano ne wakarazu yatte iu n darou ne wasuretai n dakedo', 'motto chanto itte yo wasurenai yo memo ni shite yo', 'asu 10 ji ni hoomu de machiawase to ka shiyou', 'utau wa jitte mo hana da', 'natsu ga satte mo suiboku wa setsu na', 'kuchi ni dashite', 'koi ni dashite', 'kimi ga itte', 'soshite jinsei saikou no hi', 'kim

In [8]:
for idx, seg in enumerate(rdict['segments']):
    for word in seg['words']:
        p = int(255*float(word['probability']))
        print(rgbtxt((0,0,0),(255,p,p),word['word']), end='')
    print()
    print(romaji_list[idx])

あのね 私 実は傷ついてるんだけど
ano ne watakushi jitsu wa kizutsuiteru n dakedo






ほら君が言ったこと
horakun ga itta koto
甘い
amai
考えたいと思えなくて
kangaetai to omoenakute
忘れてたんだけど
wasureteta n dakedo
盲目的に 妄想的に 妄想的に 生きて
moumokuteki ni mousouteki ni mousouteki ni ikite
衝動的な 焦燥的な 消極的なままじゃ ダメだったんだ
shoudouteki na shousouteki na shoukyokuteki na mama ja dame datta n da
きっと
kitto
人生最高の日を前に思うのだろう
jinsei saikou no hi wo mae ni omou no darou
全部全部痛いのですけど
zenbu zenbu itai no desukedo
ああ、いつか
aa, itsu ka
人生最高の日
jinsei saikou no hi
君がいないことを
kimi ga inai koto wo
もっともっともっと
motto motto motto
もっとちゃんと言って
motto chanto itte
【歌】
[uta]
あのね
ano ne
空が青いのってどうやって
sora ga aoi no tte dou yatte
伝えればいいんだろうね
tsutaereba ii n darou ne
夜の 雲が 高いのってどうすれば
yoru no kumo ga takai no tte dou sureba
君もわかるんだろう 言って
kimi mo wakaru n darou itte
あのね 私 実は分かってるの もう君が言ったこと
ano ne watakushi jitsu wa wakatteru no mou kimi ga itta koto
あのね わからずやって言うんだろうね 忘れたいんだけど
ano ne wakarazu yatte iu n darou ne wasuretai n dakedo
もっとちゃんと言ってよ 忘れないよメモにしてよ
motto chanto itte yo wa

In [25]:
org_lyrics = ''.join([pt for pt in org_ly[1] if pt.isalnum()])

In [28]:
print(gen_ly['rom_ly'])
print(org_lyrics)

anonewatakushijitsuwakizutsuiterun dakedohorakimigagentakotokanikangaeetaito omoeenakutewasureretetan dakedomoumematonimidarisoumatonimidarisoumatoninamakiteshoudoumatonaShou?matonashoukyokumatonamamajadamedatan dakittohitonamasaikounohiwomaeniOmouunodarouzenbuzenbuitainodesukedoaa,itsukahitonamasaikounohikimigainaikotowomottomottomottomottochantogentte[uta]anonesoragaaoinottedouyattetsuteerebaiin darouneyorunokumogakouinottedousurebakimimowagarun darougentteanonewatakushijitsuwabunkaterunomoukimigagentakotoanonewakarazuyattegenundarounewasureretain dakedomottochantogentteyowasurerenaiyomemonishiteyoAkirahi10jiniho- mudemachichiawasewaseto kashiyouutauwajittemohanadanatsugasattemomizusumiwasetsunakuchinideshitekoinideshitekimigagenttesoshitehitonamasaikounohikimigamierunonarakittohitonamasaigonohimoaiwoutaunodarouzenbuzenbumidesenakattattegenukaraaaitsukahitonamasaigonohikimimitainaikotogamadashinjirarenaikedomottomottomottomottomottomottokimigamottomottomottomottomottochantogentte
itt

In [29]:
sm = cp.similarity_matrix(org_lyrics, gen_ly['rom_ly'])

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'